In [ ]:
import requests
import bs4
import lxml.html as html
import pymorphy2
import string
from dask.distributed import Client
import dask.bag as db
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

url=r"http://lit.lib.ru/d/drozd_t_p/text_0240-1.shtml"

####
def pymorphy2_311_hotfix():
    from inspect import getfullargspec
    from pymorphy2.units.base import BaseAnalyzerUnit
    def _get_param_names_311(klass):
        if klass.__init__ is object.__init__:
            return []
        args = getfullargspec(klass.__init__).args
        return sorted(args[1:])
    setattr(BaseAnalyzerUnit, '_get_param_names', _get_param_names_311)

def process_text(text):
    pymorphy2_311_hotfix() # Вызов костыля
    morph = pymorphy2.MorphAnalyzer() # Создание морфологического анализатора
    words = text.lower().translate(str.maketrans('', '', string.punctuation)).translate(str.maketrans('', '', string.digits)).split() # Перевод в нижний регистр, очистка от знаков пунктуации и цифр.
    lemmatized_words = [morph.parse(word)[0].normal_form for word in words] # Лемматизация
    return [word for word in lemmatized_words if len(word) >= 3] # Оставляем слова не короче 3 символов
####

response=requests.get(url)
status_code=response.status_code
if status_code==200: # Проверяем код ответа
    soup=bs4.BeautifulSoup(response.text, features='lxml')
    text=soup.body.find('div') # Пропускаем header и footer
    [e.clear() for e in text.find_all('i')] # Поиск и удаление курсивного текста (название произведения и глав)
    text=text.get_text(' ', strip=True)  # Извлекаем оставшийся текст
    client = Client(n_workers = 6, threads_per_worker = 2, processes = True, memory_limit = '2GB') # Создание клиента DASK
    bag=db.from_sequence([text]) # Создаем объект bag
    words=bag.map(process_text).flatten() # Получаем список слов 
    word_counts=words.frequencies() # Получаем список из кортежей слов с их частотами
    df=pd.DataFrame(word_counts, columns=['Слово', 'Частота']).sort_values(by="Частота", ascending=False).head(10) # Создаем отсортированный датафрейм и берем первые 10 значений
    # Визуализация
    plt.figure(figsize=(15, 5))
    plt.title ("Наиболее часто встречающиеся слова")
    sns.barplot(data=df, x='Слово', y='Частота')
    bars = plt.bar(df['Слово'], df['Частота'])
    for bar in bars: # Добавление аннотаций
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{bar.get_height():.0f}', ha='center', va='bottom', fontsize=10)
    plt.show()
else:
    print (f"Возникла ошибка при загрузке текста. Код ошибки: {status_code}")